# PA_DPA_3-AES_DPA_Attack

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA - **NOTE: this attack is less reliable on the CWLITEXMEGA than other targets**
* CWNANO

Before starting this tutorial, it's recommended that you first complete the earlier PA_DPA tutorials since these will familiarize you with the concept of Differental Power Analysis. With that out of the way, let's look at how this attack works.

## DPA Attack Theory

As we explored in the earlier DPA tutorials, the Hamming Weight of the result of the SBox operation in AES has a measurable effect on the power consumed by the microcontroller. It turns out that just this effect (and not anything stronger, such as its linearity) is enough information to break an AES key. There's a few different ways we could go about this, but for this tutorial, we'll be looking at difference of means. With this technique, the goal is to separate the traces by a bit in the result of the SBox output (it doesn't matter which one): if that bit is 1, its group of traces should, on average, have higher power consumption during the SBox operation than the other set. 

Whether or not we get a large difference in the means between these two groups depends on whether they were properly sorted into these groups. If not, there should be, on average, little difference between the two and therefore a low difference of means. Recall the SBox operation:

![title](https://wiki.newae.com/images/7/71/Sbox_cpa_detail.png)

The SBox output depends on the subkey, which we don't know (and the plaintext, which we do). However, since there's a large difference of means for the correct key and small ones for the rest of the possible subkeys, we have a method of checking whether a given subkey is correct. If we calculate the difference of means for each subkey, the correct one will have the largest difference of means.

Our plan looks as follows
1. Capture a bunch of power traces with varying plaintext
1. Group each trace by the value of their SBox output's lowest bit for a given key guess
1. Calculate the difference of means
1. Repeat for each possible subkey
1. Select the largest difference of means -> this is the correct subkey
1. Repeat for each subkey in the key

At the end, we should get a correct AES key!

In [51]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = "NONE"#Crypto code is already tied into the simple serial program
#CRYPTO_TARGET = 'TINYAES128C'

In [52]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/willGIFTDPA
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-base-CWLITEARM.hex
rm -f -- simpleserial-base-CWLITEARM.eep
rm -f -- simpleserial-base-CWLITEARM.cof
rm -f -- simpleserial-base-CWLITEARM.elf
rm -f -- simpleserial-base-CWLITEARM.map
rm -f -- simpleserial-base-CWLITEARM.sym
rm -f -- simpleserial-base-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-base.s crypto.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- simpleserial-base.d crypto.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- simpleserial-base.i crypto.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simpl

## Capturing Power Traces

Capture and setup is similar to earlier tutorials. We'll have to capture a fair number of traces (usually a few thousand here) since Difference of Means isn't a super trace efficient method. As you'll find during the CPA tutorials, CPA is much better in this regard - it can often break AES implementations such as these in under 50 traces.

You may also find that you need to modify gain settings and the number of traces you capture - this attack is much more sensitive to gain settings and noise than a CPA attack would be. 

### Setup

In [53]:
#scope.dis()
#target.dis()
%run "Helper_Scripts/Setup_Generic.ipynb"

Serial baud rate = 38400


In [54]:
fw_path = "../hardware/victims/firmware/willGIFTDPA/simpleserial-base-{}.hex".format(PLATFORM)

In [55]:
cw.program_target(scope, prog, fw_path)

Serial baud rate = 115200
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5455 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5455 bytes
Serial baud rate = 38400


### Capture

In [87]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()

traces = []
N = 150000  # Number of traces

if PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    scope.adc.samples = 12000
    #scope.adc.offset = 12500
    
    #scope.gain.db = 10
    scope.gain.db = 34
elif PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    scope.gain.db = 34 #works best with this gain for some reason
    scope.adc.samples = 1700 - 170
    scope.adc.offset = 500 + 700 + 170
    N = 5000
    
print(scope)
for i in tnrange(N, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    from binascii import hexlify
    if(i == 0):
        print("key", hexlify(key))

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    traces.append(trace)

#Convert traces to numpy arrays
trace_array = np.asarray([trace.wave for trace in traces])
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])  # for fixed key, these keys are all the same
textout_array = np.asarray([trace.textout for trace in traces])

#for i in range(20):
#    print (textout_array[i])
    

cwlite Device
gain = 
    mode = high
    gain = 44
    db   = 33.859375
adc = 
    state      = False
    basic_mode = rising_edge
    timeout    = 2
    offset     = 0
    presamples = 0
    samples    = 12000
    decimate   = 1
    trig_count = 183340
clock = 
    adc_src       = clkgen_x4
    adc_phase     = 0
    adc_freq      = 29538448
    adc_rate      = 29538448.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1       = serial_rx
    tio2       = serial_tx
    tio3       = high_z
    tio4       = high_z
    pdid       = high_z
    pdic       = high_z
    nrst       = high_z
    glitch_hp  = False
    glitch_lp  = False
    extclk_src = hs1
    hs2        = clkgen
    target_pwr = True
glitch = 
    clk_src     = target
    wi

key b'2b7e151628aed2a6abf7158809cf4f3c'


## Analysis

As we discussed above, our goal here is to find the biggest difference of means out of the possible subkey values we could have. First, we'll get some values and functions that will be useful for our calculations. We'll be using `intermediate()` later to get the output of the SBox from a plaintext and key input.

In [88]:
numtraces = np.shape(trace_array)[0] #total number of traces
numpoints = np.shape(trace_array)[1] #samples per trace
print(numtraces)

sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

GIFTSbox = ( 0x1, 0xa, 0x4, 0xc, 0x6, 0xf, 0x3, 0x9, 0x2, 0xd, 0xb, 0x7, 0x5, 0x0, 0x8, 0xe)

GIFTPbox64 =             ( 0,  17, 34, 51, 48, 1,  18, 35, 32, 49, 2,  19, 16,
                           33, 50, 3,  4,  21, 38, 55, 52, 5,  22, 39, 36, 53,
                           6,  23, 20, 37, 54, 7,  8,  25, 42, 59, 56, 9,  26,
                           43, 40, 57, 10, 27, 24, 41, 58, 11, 12, 29, 46, 63,
                           60, 13, 30, 47, 44, 61, 14, 31, 28, 45, 62, 15 )

#print( GIFTSbox[0])

def BitToByte(input):
    return (int)((input)/8)

def NibbleToByte(input):
    return (int)((input)/2)

def ByteInputSBoxGIFT(input):
    temp = GIFTSbox[(input & 0xf)]
    temp2 = GIFTSbox[((input >> 4) & 0xf)]
    return (temp2 << 4) | temp

def ApplySBox64(input):
    for i in range(8):
        input[i] = ByteInputSBoxGIFT(input[i])
    return input

def ApplyPLayer(input):
    #print ("player function input " , input)
    temp = []
    for i in range(8):
        temp.append(input[i])
        temp[i] = 0 #Used to create an empty byte array of length 8
        #print("byte " , i , " has value " , input[i] )
    for i in range(64):
        tempBit = ((input[BitToByte(i)]) >> ((i%8))) & 0x1
        #print (tempBit)
        pVal = GIFTPbox64[i]
        temp[BitToByte(pVal)] |= (tempBit << (pVal%8))
    return temp

def Invert(input):
    temp = []
    for i in range(8):
        temp.append(input[7-i])
    return temp

def GIFT64PTtoIntermediate1(pt):  #value inbetween 1st pLayer and 1st Add round key
    #temp = Invert(pt)
    temp = []
    for i in range(8):
        temp.append(pt[i])
    temp = ApplySBox64(temp)
    temp = ApplyPLayer(temp)
    return temp

def GIFT64PTtoIntermediate2(pt, RK1):
    temp = GIFT64PTtoIntermediate(pt)
    for i in range(8):
        temp[i] = temp[i] ^ RK1[i]
    temp = GIFT64PTtoIntermediate(temp)
    return temp

def GIFT64PTtoIntermediate3(pt, RK1, RK2):
    temp = GIFT64PTtoIntermediate2(pt, RK1)
    for i in range(8):
        temp[i] = temp[i] ^ RK2[i]
    temp = GIFT64PTtoIntermediate(temp)
    return temp

def GIFT64PTtoIntermediate4(pt, RK1, RK2, RK3):
    temp = GIFT64PTtoIntermediate3(pr, RK1, RK2)
    for i in range(8):
        temp[i] = temp[i] ^ RK3[i]
    temp = GIFT64PTtoIntermediate(temp)
    return temp

def applyGuess(index, buffer, subKeyGuess):
    #print(NibbleToByte(index))
    temp = (buffer[NibbleToByte(index)] >> ((index % 2)*4)) & 0x0f
    temp = temp ^ subKeyGuess
    temp = GIFTSbox[temp]
    #print(temp)
    #temp = buffer[0] & 0xf
    #temp = temp ^ subKeyGuess
    #temp = GIFTSbox[temp]
    return temp
    

150000


Our first step here will be separating our traces into different groups based on the SBox's output. As mentioned earlier, we're separating based on the least significant bit, but really any bit would work (as a test, you can change this and see if the attack still works):

```Python
one_list = []
zero_list = []
for tnum in range(numtraces):
    if (intermediate(textin_array[tnum][subkey], kguess) & 1):
        one_list.append(trace_array[tnum])
    else:
        zero_list.append(trace_array[tnum])
```

Then calculate the difference of means:

```Python
one_avg = np.asarray(one_list).mean(axis=0)
zero_avg = np.asarray(zero_list).mean(axis=0)
mean_diffs[kguess] = np.max(abs(one_avg - zero_avg))
```

We'll need to repeat this with each possible key guess and then pick the one with the highest difference of means:
```Python
guess = np.argsort(mean_diffs)[-1]
key_guess.append(guess)
print(hex(guess))
print(mean_diffs[guess])
```

Finally, altogether and attacking all of the subkeys:

In [89]:
from tqdm import tnrange
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

xrange = range(len(traces[0].wave))
p.line(xrange, traces[0].wave, line_color="red")
show(p)

mean_diffs = np.zeros(16)
key_guess = []
known_key = known_keys[0]
plots = []
#print("Intermedartary state for trace 1 " , GIFT64PTtoIntermediate1(textin_array[6]))
#print("Intermedartary state for trace 2 " , GIFT64PTtoIntermediate1(textin_array[10]))
for i in range(10):
    print (textin_array[i])
for subkey in tnrange(0, 16, desc="Attacking Subkey"):
    for kguess in tnrange(16, desc="Keyguess", leave=False):
        one_list = []
        zero_list = []
        
        tempTestArr = []
        tempTestArr2 = []
        
        for tnum in range(numtraces):
            #if (intermediate(textin_array[tnum][subkey], kguess) & 1): #LSB is 1
            tempNum = GIFT64PTtoIntermediate1(textin_array[tnum])
            #print (tempNum)
            #tempNum = Invert(tempNum)
            appliedGuess = applyGuess(subkey, tempNum, kguess)
            #print ("applied guess " , appliedGuess)
            if(tnum < 10):
                #print("temp num " , tempNum[0], "  keyGuess " , kguess)
                #tempTestArr.append(appliedGuess & 0x1)
                tempTestArr.append(appliedGuess)
            #if(appliedGuess == 0xf):
            #    one_list.append(trace_array[tnum])
            #if(appliedGuess == 0x0):
            #   zero_list.append(trace_array[tnum])
            if( ((appliedGuess >> 3) % 2) == 1):
                one_list.append(trace_array[tnum])
            else:
                zero_list.append(trace_array[tnum])
        #print ("here")
        #print("size of 1 list " , len(one_list))
        #print("size of 0 list " , len(zero_list))
        one_avg = np.asarray(one_list).mean(axis=0)
        #print("guess " , kguess , " onv_avg = ", one_avg)
        zero_avg = np.asarray(zero_list).mean(axis=0)
        #print("zero_avg : " , zero_avg)
        mean_diffs[kguess] = np.max(abs(one_avg - zero_avg))
        #print("subkey guess ", kguess , " has value of " , mean_diffs[kguess])
        #print("Test Arr:" , tempTestArr)
        #print("length of 1 list ", len(one_list))
        #print("length of 0 list " , len(zero_list))
        #print("Intermedartary state for trace 1 " , GIFT64PTtoIntermediate1(textin_array[0]))
        if kguess == known_key[subkey]:
            plots.append(abs(one_avg - zero_avg))
    guess = np.argsort(mean_diffs)[-1]
    print("guess " , hex(guess), " for subkey " , subkey)
    for i in range(16):
        print ("guess " , hex(i) , " has value " , mean_diffs[i])
    key_guess.append(guess)
    #print(hex(guess) + "(real = 0x{:02X})".format(known_key[subkey]))
    #mean_diffs.sort()
    #print(mean_diffs[guess])
    #print(mean_diffs[known_key[subkey]])
print ("key guess : " , key_guess )

Loading BokehJS ...

[173 188  96  73 252 117 231 251 135 209  84  41 224  72 156 194]
[ 45 204 111 202 205  86 179  56  70  15  27 117 124 240  50 189]
[106 199 176 242  16 157 243 174  85 183  44 237  24 133  90 231]
[226 204 143 233 108 235 129  74  56 220 108 118  36 167 166 196]
[154 169  52 113 243 159 220 170  92  90  42 165 142  47 208 180]
[106 162 186 240  38 255  89 204   4 200  29  49   1  45 172  58]
[186  96  62 219 199 147 217  14 215  13  52  93 113  13 244 178]
[109 211 154  24  86 111 226 179 215  40 252 191 207  49  43 190]
[175 125   8  12  43 216  99 104 232 205 233  29  85 231 192 245]
[ 89 182 174 103 105  19  32 113 111 241 200 145 102 238  49  12]


guess  0xb  for subkey  0
guess  0x0  has value  0.025409739032896672
guess  0x1  has value  0.024288484351273565
guess  0x2  has value  0.023512118019819617
guess  0x3  has value  0.024633672122174044
guess  0x4  has value  0.024394214334325204
guess  0x5  has value  0.02315980266861395
guess  0x6  has value  0.024527805510023037
guess  0x7  has value  0.02576199598716916
guess  0x8  has value  0.0358547567729921
guess  0x9  has value  0.033215526436860876
guess  0xa  has value  0.03166307865194817
guess  0xb  has value  0.037407321223172196
guess  0xc  has value  0.023009287718143256
guess  0xd  has value  0.021907689038945855
guess  0xe  has value  0.022623092844380355
guess  0xf  has value  0.023724999518929646


guess  0x1  for subkey  1
guess  0x0  has value  0.035127914850952985
guess  0x1  has value  0.0381465582181858
guess  0x2  has value  0.03673309288604404
guess  0x3  has value  0.036541322699943285
guess  0x4  has value  0.023393984953683805
guess  0x5  has value  0.023553949828521187
guess  0x6  has value  0.02361740650750091
guess  0x7  has value  0.02345751244085978
guess  0x8  has value  0.02374578802022098
guess  0x9  has value  0.02417806543755674
guess  0xa  has value  0.02427965700445811
guess  0xb  has value  0.023847944948827582
guess  0xc  has value  0.02440531490328185
guess  0xd  has value  0.025026543933643203
guess  0xe  has value  0.023620057042819442
guess  0xf  has value  0.02299905093082172


guess  0x2  for subkey  2
guess  0x0  has value  0.033884382702353055
guess  0x1  has value  0.03620258481082417
guess  0x2  has value  0.03723434017282301
guess  0x3  has value  0.03285255846918875
guess  0x4  has value  0.024387411177438922
guess  0x5  has value  0.0217367755705771
guess  0x6  has value  0.02226694893800471
guess  0x7  has value  0.02491764230224311
guess  0x8  has value  0.0230239953988238
guess  0x9  has value  0.02325648325626592
guess  0xa  has value  0.025282031176186354
guess  0xb  has value  0.025049708276755234
guess  0xc  has value  0.02435768175067493
guess  0xd  has value  0.024450958487876873
guess  0xe  has value  0.023948291861881382
guess  0xf  has value  0.023855014326683383


...

guess  0x1  for subkey  11
guess  0x0  has value  0.03505469161554864
guess  0x1  has value  0.03862186689248687
guess  0x2  has value  0.036783377133508544
guess  0x3  has value  0.03689231218086364
guess  0x4  has value  0.02301133016532142
guess  0x5  has value  0.02479336444123134
guess  0x6  has value  0.025446521732846755
guess  0x7  has value  0.023664776073317434
guess  0x8  has value  0.022836594112614794
guess  0x9  has value  0.022466081217171058
guess  0xa  has value  0.021246441549249095
guess  0xb  has value  0.021617043535443803
guess  0xc  has value  0.021582334092106303
guess  0xd  has value  0.021420957825411924
guess  0xe  has value  0.02250070256284295
guess  0xf  has value  0.02266212173003304


guess  0x1  for subkey  12
guess  0x0  has value  0.041664957101838096
guess  0x1  has value  0.04176025751287554
guess  0x2  has value  0.04119851591652718
guess  0x3  has value  0.04110262502334594
guess  0x4  has value  0.041613785298031836
guess  0x5  has value  0.04170052821325437
guess  0x6  has value  0.04124917078748924
guess  0x7  has value  0.041162297762740385
guess  0x8  has value  0.01390953699706754
guess  0x9  has value  0.019136130922363215
guess  0xa  has value  0.0185179977265576
guess  0xb  has value  0.014527526575843691
guess  0xc  has value  0.008533361916996773
guess  0xd  has value  0.005899193451214979
guess  0xe  has value  0.006018050063616609
guess  0xf  has value  0.008934378699485324


guess  0x1  for subkey  13
guess  0x0  has value  0.04357894726639361
guess  0x1  has value  0.043581744836054925
guess  0x2  has value  0.04184670328875051
guess  0x3  has value  0.04184425596214775
guess  0x4  has value  0.041887651465491116
guess  0x5  has value  0.04225581751998045
guess  0x6  has value  0.043538353047558015
guess  0x7  has value  0.04317138702591286
guess  0x8  has value  0.011615103585177311
guess  0x9  has value  0.01025984069633376
guess  0xa  has value  0.010759744682700856
guess  0xb  has value  0.011115151307532517
guess  0xc  has value  0.007857496712936896
guess  0xd  has value  0.007201345954076399
guess  0xe  has value  0.007269118340384184
guess  0xf  has value  0.007834036940636502


guess  0x1  for subkey  14
guess  0x0  has value  0.041127819648970776
guess  0x1  has value  0.04288096497344818
guess  0x2  has value  0.04260148258869659
guess  0x3  has value  0.04084865327930595
guess  0x4  has value  0.0414844298728381
guess  0x5  has value  0.04268287127790654
guess  0x6  has value  0.042245018967247444
guess  0x7  has value  0.04104747292668712
guess  0x8  has value  0.015865072679556724
guess  0x9  has value  0.01569651352187737
guess  0xa  has value  0.014683047229612778
guess  0xb  has value  0.016878609489152763
guess  0xc  has value  0.00797011855180968
guess  0xd  has value  0.007202862969731327
guess  0xe  has value  0.0072460804527810785
guess  0xf  has value  0.008364343569080446


KeyboardInterrupt: 

With that done, we should now have the correct key:

In [90]:
for i in range(16):
        print ("guess " , hex(i) , " has value " , mean_diffs[i])
print(key_guess)
print(known_key)

guess  0x0  has value  0.04254152248751833
guess  0x1  has value  0.042457141338855836
guess  0x2  has value  0.04236567511185271
guess  0x3  has value  0.04245022946965207
guess  0x4  has value  0.04332469528282393
guess  0x5  has value  0.042944615394802016
guess  0x6  has value  0.041582331572582176
guess  0x7  has value  0.041963152072819615
guess  0x8  has value  0.03885771452253789
guess  0x9  has value  0.036360330856935436
guess  0xa  has value  0.014683047229612778
guess  0xb  has value  0.016878609489152763
guess  0xc  has value  0.00797011855180968
guess  0xd  has value  0.007202862969731327
guess  0xe  has value  0.0072460804527810785
guess  0xf  has value  0.008364343569080446
[11, 1, 2, 1, 7, 4, 4, 0, 0, 0, 3, 1, 1, 1, 1]
[ 43 126  21  22  40 174 210 166 171 247  21 136   9 207  79  60]


We can also plot the difference of means for a few of the correct subkey bytes:

In [10]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()
p.line(range(numpoints), plots[0], line_color='green')
p.line(range(numpoints), plots[1], line_color='red')
p.line(range(numpoints), plots[15], line_color='blue')
show(p)

Loading BokehJS ...

## Conclusion

Congratulations, you have (hopefully) broken AES using a DPA attack! As you might have discovered during this tutorial, there can be quite a few issues with the difference of means method for breaking AES keys:

* It's quite susceptible to noise
* The attack can easily pick up other parts of the AES operation
* The attack typically requires a lot of traces. These software AES implementations are pretty weak against power analysis, but they still required thousands of traces to break
* Some targets (such as the XMega) require fine tuning settings to make the attack work

Nevertheless, using a difference of means attack can still be very useful. For example, a later tutorial, PA_Multi_1, uses a difference of means attack similar in concept to this one to break the signature of an AES256 bootloader.

## Tests

In [50]:
assert (known_key == key_guess).all(), "Failed to break key.\nGot: {}\nExp: {}".format(key_guess, known_key)

AssertionError: Failed to break key.
Got: [43, 156, 247, 151, 69, 174, 51, 39, 114, 118, 148, 9, 80, 46, 142, 189]
Exp: [ 43 126  21  22  40 174 210 166 171 247  21 136   9 207  79  60]